# Tarea Apache Spark
>Importa las librerías necesarias dónde sea necesario


In [1]:
import pandas as pd 
from pyspark.sql import SparkSession

### SparkSession
>Crea un SparkSession para comenzar la tarea

In [5]:
spark = SparkSession.builder.appName("Miapp").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/23 00:11:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Crear un DataFrame
>Lee el csv datosTarea.csv, mételo a un DF y muéstralo.

In [7]:
# Leer el archivo CSV y cargarlo en un DataFrame
ruta='notebooks/Tarea/datosTarea.csv'

df = spark.read.csv('datosTarea.csv', header=True, inferSchema=True)
df.show()

+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+--------+-----------+------+
|Index|Organization Id|                Name|             Website|             Country|         Description|Founded|            Industry|Number of employees|Networth|stock_price|profit|
+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+--------+-----------+------+
|    1|FAB0d41d5b5d22c|         Ferrell LLC|  https://price.net/|    Papua New Guinea|Horizontal empowe...|   1990|            Plastics|               3498|  402269|         33| 12125|
|    2|6A7EdDEA9FaDC52|Mckinney, Riley a...|http://www.hall-b...|             Finland|User-centric syst...|   2015|Glass / Ceramics ...|               4952|  569480|         49| 12001|
|    3|0bFED1ADAE4bcC1|          Hester Ltd|http://sullivan-r...|          

### Filtro de datos
>Consigue todas las empresas que empiecen con 'M' y tengan entre 4000 y 7000 empleados. Sólo muestra los nombres y el número de empleados.

In [9]:
from pyspark.sql.functions import col
resultados = df.filter((col("Name").startswith("M")) & (col("Number of employees").between(4000, 7000)))
# Seleccionar solo las columnas requeridas
resultados = resultados.select("Name", "Number of employees")
# Mostrar los resultados
resultados.show())

+--------------------+-------------------+
|                Name|Number of employees|
+--------------------+-------------------+
|Mckinney, Riley a...|               4952|
|       Mcintosh-Mora|               4389|
|     Mckenzie-Melton|               4589|
|          Massey LLC|               5004|
|        Mays-Preston|               5786|
+--------------------+-------------------+



>Consigue todos los países que no inicien con las letras 'b', 's' y 'm', pero que tampoco tengan un netword mayor a 500000. Muestra el nombre de la compañía, el país y el networth.

In [10]:
# Realizar el filtro de datos
resultados = df.filter(
    (~col("Country").startswith("b")) & 
    (~col("Country").startswith("s")) & 
    (~col("Country").startswith("m")) & 
    (col("Networth") <= 500000)
)

# Seleccionar solo las columnas requeridas
resultados = resultados.select("Name", "Country", "Networth")

# Mostrar los resultados
resultados.show()

+--------------------+--------------------+--------+
|                Name|             Country|Networth|
+--------------------+--------------------+--------+
|         Ferrell LLC|    Papua New Guinea|  402269|
|      Holder-Sellers|        Turkmenistan|  105914|
|Keller, Campos an...|             Liberia|  329130|
|      Pacheco-Spears|              Sweden|   88435|
|         Harrell LLC|          Guadeloupe|  251274|
|         Jenkins Inc|        South Africa|  139725|
|Dickson, Richmond...|      Czech Republic|  359030|
|        Prince-David|    Christmas Island|  120289|
|         Rivas Group|           Australia|  477824|
|Sloan, Mays and W...|                Chad|   41975|
|Glass, Barrera an...|     Kyrgyz Republic|  300150|
|          Pineda-Cox|             Bolivia|  150880|
|Baker, Mccann and...|               Kenya|  188370|
|            Hahn PLC|             Belarus|  427224|
|Valentine, Fergus...|              Jersey|  412274|
|           Walls LLC|          Cape Verde|  1

### Funciones
Crea una función con @pandas_udf que que le reste a los profits la media en cada renglón. Crea una nueva columna que muestre los resultados.

In [11]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

# Calcular la media de la columna 'profits'
mean_profit = df.select(col("profit")).groupBy().avg().first()[0]

# Crear una UDF para restar la media de la columna 'profit'
@udf(DoubleType())
def subtract_mean(value):
    return value - mean_profit

# Aplicar la UDF al DataFrame para crear una nueva columna
df_with_subtracted_mean = df.withColumn("profit_minus_mean", subtract_mean(col("profit")))

# Muestra los resultados
df_with_subtracted_mean.show()

+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+--------+-----------+------+-------------------+
|Index|Organization Id|                Name|             Website|             Country|         Description|Founded|            Industry|Number of employees|Networth|stock_price|profit|  profit_minus_mean|
+-----+---------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------------------+--------+-----------+------+-------------------+
|    1|FAB0d41d5b5d22c|         Ferrell LLC|  https://price.net/|    Papua New Guinea|Horizontal empowe...|   1990|            Plastics|               3498|  402269|         33| 12125|-248.64999999999964|
|    2|6A7EdDEA9FaDC52|Mckinney, Riley a...|http://www.hall-b...|             Finland|User-centric syst...|   2015|Glass / Ceramics ...|               4952|  569480|         49| 12

### Grouping data
>Agrupa por industry y muestra cuáles son las empresas con el profit más alto. Muestra los primeros tres.

In [12]:
from pyspark.sql.functions import max, avg

# Agrupar por 'industry' y encontrar la empresa con el profit más alto en cada industria
top_profits_by_industry = df.groupBy("Industry").agg(max("profit").alias("MaxProfit"))

# Ordenar por 'MaxProfit' en orden descendente y mostrar las tres primeras
top_profits_by_industry.orderBy(col("MaxProfit").desc()).show(3)

+--------------------+---------+
|            Industry|MaxProfit|
+--------------------+---------+
|  Legislative Office|    19363|
|Museums / Institu...|    19079|
|             Farming|    18850|
+--------------------+---------+
only showing top 3 rows



>Agrupa por industry y calcula el promedio de empleados que tienen

In [13]:
# Agrupar por 'industry' y calcular el promedio de empleados
average_employees_by_industry = df.groupBy("Industry").agg(avg("Number of employees").alias("AverageEmployees"))

# Mostrar los resultados
average_employees_by_industry.show()

+--------------------+-----------------+
|            Industry| AverageEmployees|
+--------------------+-----------------+
|Primary / Seconda...|6457.666666666667|
|     Broadcast Media|           2589.0|
|           Wholesale|           5010.0|
|Investment Manage...|           3133.5|
|    Food / Beverages|           9011.0|
|  Gambling / Casinos|           4873.0|
|Logistics / Procu...|           4155.0|
|            Maritime|            769.0|
|            Wireless|           6146.0|
|Education Management|            339.0|
|       Arts / Crafts|           2800.0|
|           Insurance|           1215.0|
|  Financial Services|           5157.0|
|Business Supplies...|           9097.0|
|Consumer Electronics|           5022.0|
|       Public Safety|           5287.0|
|Information Techn...|           3934.0|
|Civic / Social Or...|           2442.0|
|      Consumer Goods|           9069.0|
|Glass / Ceramics ...|           4952.0|
+--------------------+-----------------+
only showing top

### SQL
>Usando Spark SQL, obtén cuántas empresas se fundaron despúes del 2000.

In [14]:
# Registrar el DataFrame como una vista temporal SQL
df.createOrReplaceTempView("empresas")

# Ejecutar la consulta SQL para obtener el número de empresas fundadas después del 2000
consulta = """SELECT COUNT(*) as total_empresas_post_2000 FROM empresas WHERE Founded > 2000"""

# Ejecutar la consulta y mostrar los resultados
res = spark.sql(consulta)
res.show()

+------------------------+
|total_empresas_post_2000|
+------------------------+
|                      38|
+------------------------+



### ML Regresión Lineal
>Con número de empleados, networth y stock price, obtén una predicción del profit a través de una regresión lineal.

In [15]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Asignar los nombres de las columnas a variables para fácil referencia
number_of_employees_col = 'Number of employees'
networth_col = 'Networth'
stock_price_col = 'stock_price'
profit_col = 'profit'

# Preparar los datos para el modelo de regresión lineal
assembler = VectorAssembler(inputCols=[number_of_employees_col, networth_col, stock_price_col], outputCol="features")
data = assembler.transform(df)

# Definir la columna de características y la columna objetivo
data = data.withColumnRenamed(profit_col, "label").select("features", "label")

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
train_data, test_data = data.randomSplit([0.7, 0.3])

# Inicializar el modelo de regresión lineal
lr = LinearRegression()

# Entrenar el modelo con los datos de entrenamiento
lr_model = lr.fit(train_data)

# Predecir los profits en el conjunto de prueba
predictions = lr_model.transform(test_data)

# Evaluar el modelo
evaluator = RegressionEvaluator(metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("RMSE:", rmse)

23/11/23 00:34:00 WARN Instrumentation: [78549413] regParam is zero, which might cause numerical instability and overfitting.
23/11/23 00:34:01 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/11/23 00:34:01 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


RMSE: 3767.034572207998


>Una vez que obtengas los resultados, a través del api de pandas, conviértelo en un pandas on spark DataFrame y pásalo a csv.

In [16]:
# Convertir el DataFrame de Spark a un DataFrame de Pandas
pandas_df = predictions.select("features", "label", "prediction").toPandas()

# Guardar el DataFrame de Pandas como CSV
pandas_df.to_csv("predictions.csv", index=False)